In [11]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [12]:
batch_size = 32
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20

In [14]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [15]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [16]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [17]:
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 11s 222us/step - loss: 1.6532 - acc: 0.4037 - val_loss: 1.4503 - val_acc: 0.4659
Epoch 2/10
50000/50000 [==============================] - 9s 184us/step - loss: 1.3523 - acc: 0.5189 - val_loss: 1.4067 - val_acc: 0.5200
Epoch 3/10
50000/50000 [==============================] - 9s 182us/step - loss: 1.2023 - acc: 0.5757 - val_loss: 1.1490 - val_acc: 0.5944
Epoch 4/10
50000/50000 [==============================] - 9s 185us/step - loss: 1.0900 - acc: 0.6188 - val_loss: 1.1293 - val_acc: 0.6014
Epoch 5/10
50000/50000 [==============================] - 9s 182us/step - loss: 0.9995 - acc: 0.6525 - val_loss: 1.0055 - val_acc: 0.6524
Epoch 6/10
50000/50000 [==============================] - 9s 185us/step - loss: 0.9276 - acc: 0.6800 - val_loss: 0.9915 - val_acc: 0.6575
Epoch 7/10
50000/50000 [==============================] - 9s 184us/step -

In [18]:
import keras2onnx
import onnxruntime

In [19]:
onnx_model = keras2onnx.convert_keras(model, model.name)

Instructions for updating:
Use tf.compat.v1.graph_util.remove_training_nodes
using tensorflow=1.13.1, onnx=1.4.1, opset=9, tfonnx=1.5.0/82f805


In [21]:
import onnx
temp_model_file = 'cifar10.onnx'
onnx.save_model(onnx_model, temp_model_file)